# АДБМ 2023-2024, 

# Методы Моделирования Пространственной структуры Протеинов

# Семинар 1: BioPython tutorial

## 1. Installation / import

In [ ]:
! pip3.8 install biopython

## 2. Модуль PDB (Работа с файлами из PDB - Protein Data Bank)

In [ ]:
import Bio.PDB as pdb

### 2.1. Открытие файла PDB, структура

In [ ]:
# Любой файл PDB может быть получен из базы с помощью указания его ID.
# Для этого используется объект класса PDBList:

from os.path import exists

pdb_list = pdb.PDBList()
for pdb_id in ["1FSD"]:
    print("Fetching {}...".format(pdb_id))
    
    # Функция возвращает путь к скачанному файлу.
    path = pdb_list.retrieve_pdb_file(pdb_id, 
                                      pdir=".",
                                      file_format="pdb")
    
    # ВАЖНО: Если белка с таким ID нет в базе, функция все равно вернет путь ¯\_(ツ)_/¯
    flag = exists(path)
    if flag:
        print("\tALL OK, file exists")
    else:
        print("\tno file ¯\_(ツ)_/¯")

In [ ]:
# Для открытия файла с белком используется объект класса PDBParser():
parser = pdb.PDBParser()
struct = parser.get_structure("_", # id структуры. может быть любым, 
                                    # но несколько структур не могут иметь тот же id
                              path)

### Посмотрим на заголовок:

In [ ]:
print(struct.header)

### Разберем, как записан белок в этом файле. Белок хранится как иерархичная структура, которая имеет следующие уровни: 
### Structure/Model/Chain/Residue/Atom.
### Как правило, у белковых структур в Structure есть только один дочерний элемент Model, но для объектов NMR-структур (Nuclear Magnetic Resonance) их может быть несколько. Дочерние элементы Model - Chain (цепочки аминокислотных остатков (Residue)), следующий уровень - сами остатки.

In [ ]:
for model in struct:
    print("Model {}:".format(model.id))
    for chain in model:
        print("\tChain {}:".format(chain.id))
        for residue in chain:
            print("\t\tResidue {}:".format(residue.id))
            for atom in residue.get_atoms():
                print("\t\t\t", atom)

### Также все аминокислоты и все атомы можно получить с помощью следующих методов:
### print(struct.get_residues())
### print(struct.get_atoms())

### В BioPython методы вида .get_* (для получения набора объектов) возвращают итерируемый, но не индексируемый объект. Чтобы работать с запрашиваемыми элементами как с массивом, нужно либо преобразовать вывод функции к структуре list:

In [ ]:
print('Для сравнения:')
residues = struct.get_residues()
print('Результат get_residues:', residues)
print('Результат преобразования к list:', list(residues))

### либо вызвать метод .get_list() у родительского типа в иерархии Structure/Model/Chain/Residue/Atom:

In [ ]:
residue = list(struct.get_residues())[0]
print('Атомы этого Residue:', residue.get_list())

### Пожалуй, второй способ предпочтительнее: в первом случае мы получаем все объекты целевого уровня, не обращая внимания на предыдущие. В результате мы можем получить, например, атомы всех цепочек, хотя нам нужна лишь одна   ¯\\_(ツ)_/¯

### Аналогично, можно получить родительский элемент иерархии с помощью метода .get_parent():

In [ ]:
print('Родительский элемент Residue (Chain):', residue.get_parent())

### 2.2. Работа с остатками и атомами

#### 2.2.1. Остатки

In [ ]:
residue = list(struct.get_residues())[0]
print('Обозначение аминокислоты:', residue.get_resname())
print('Атомы остатка:', residue.get_list())
print('Получение атома остатка по его обозначению:', residue['CG'])

#### 2.2.2. Атомы

In [ ]:
atom = residue.get_list()[5]
print('Объект Atom', atom)
print('Обозначение атома:', atom.get_fullname())
print('Химический элемент:', atom.element)
print('Координаты атома (тип Vector):', atom.get_vector())
print('Координаты атома (тип list):', atom.get_coord())
print('Масса атома:', atom.mass)

### Некоторые поля, получаемые с помощью методов .get_*, также могут быть получены с помощью обращения к полям объекта:

In [ ]:
print(atom.get_fullname())
print(atom.fullname)

### 2.3. Вычисления

<img src="https://www.webmo.net/link/help/img/blbada.jpg"></img>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cg = residue['CG']
cb = residue['CB']
ca = residue['CA']
n = residue['N']

#### 2.3.1 Расстояние между атомами вычисляется как их разность (порядок не важен):

In [ ]:
print('Расстояние между', cg.fullname, 'и', cb.fullname, 'равно', cg - cb)

#### Пример: построение матрицы расстояний / контактов

In [ ]:
residues = struct[0].child_list[0].child_list
matr = np.zeros((len(residues), len(residues)))

# your code here

R = 8

_, axs = plt.subplots(1,2,figsize=(14,7))
axs[0].imshow(matr)
axs[1].imshow(matr < R)
plt.show()

#### 2.3.2. Планарные углы

In [ ]:
angle = pdb.calc_angle(cg.get_vector(), cb.get_vector(), ca.get_vector())
print('Угол CA-CB-CG равен {} ({} градусов)'.format(angle, angle/np.pi*180))

#### 2.3.3. Двугранные (торсионные) углы

In [ ]:
chi1 = pdb.calc_dihedral(n.get_vector(), ca.get_vector(), cb.get_vector(), cg.get_vector())
print('Угол chi1 равен', chi1/np.pi*180, 'градусов')

#### Пример: построение карты Рамачандрана

In [ ]:
ang_pairs = np.empty((0,2))

# your code here

plt.figure(figsize=(7,7))
plt.scatter(*ang_pairs.T)
plt.plot([-180, -180, 180, 180, -180], [-180, 180, 180, -180, -180], c="black", linewidth=1)
plt.plot([0,0], [-180,180], c="black", linewidth=1)
plt.plot([-180,180], [0,0], c="black", linewidth=1)
plt.xticks(np.arange(-180, 181, 90))
plt.yticks(np.arange(-180, 181, 90))
plt.xlabel("$\phi$", fontsize=14)
plt.ylabel("$\psi$", fontsize=14)

plt.show()